In [1]:
 !pip install pycaret==1.0
 !pip install scipy==1.2 --upgrade

  Using cached scikit_learn-0.22-cp37-cp37m-win_amd64.whl (6.2 MB)
  Using cached scipy-1.7.3-cp37-cp37m-win_amd64.whl (34.1 MB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.2.0
    Uninstalling scipy-1.2.0:
      Successfully uninstalled scipy-1.2.0


ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'C:\\Users\\wew12\\AppData\\Local\\Temp\\pip-uninstall-1lnbxi1c\\sparse\\csgraph\\_min_spanning_tree.cp37-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



  Using cached scipy-1.2.0-cp37-cp37m-win_amd64.whl (31.7 MB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
statsmodels 0.13.5 requires scipy<1.8,>=1.3; python_version == "3.7", but you have scipy 1.2.0 which is incompatible.
statsmodels 0.13.5 requires scipy>=1.3; (python_version > "3.9" or platform_system != "Windows" or platform_machine != "x86") and python_version < "3.12", but you have scipy 1.2.0 which is incompatible.
scikit-image 0.19.3 requires scipy>=1.4.1, but you have scipy 1.2.0 which is incompatible.
pyod 1.0.6 requires scipy>=1.5.1, but you have scipy 1.2.0 which is incompatible.
pycaret 1.0.0 requires scikit-learn==0.22, but you have scikit-learn 0.23.2 which is incompatible.
phik 0.12.2 requires scipy>=1.5.2, but you have scipy 1.2.0 which is incompatible.
optuna 3.0.3 requires scipy<1.9.0,>=1.7.0, but you have scipy 1.2.0 which is incompatible.
mlxtend 0.19.0 requires scipy>=1.2.1, but you have scipy 1

In [2]:
import random
import os
import numpy as np


# 시드 정하기!!!!!!!

CFG = {
    'SEED':42,
    'NSPLIT' :10

}

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(CFG['SEED']) # Seed 고정

In [3]:
import pandas as pd
import numpy as np

train_df = pd.read_csv(r'C:\A\breastCancer\ml/train.csv')
test_df = pd.read_csv(r"C:\A\breastCancer\ml/test.csv")


In [4]:
# mean, fillna(0)
train_df['size of cancers'] = train_df['size of cancers'].fillna(train_df['size of cancers'].mean())
train_df = train_df.fillna(0)

test_df['size of cancers'] = test_df['size of cancers'].fillna(train_df['size of cancers'].mean())
test_df = test_df.fillna(0)


target_name = ['N_category']

# 명목형 변수
cat_col_names = [
    # 기본
    #'dx','location of cancers', 'number of cancers', 'NG','HG','HG_score_1','HG_score_2','HG_score_3','DCIS_or_LCIS','DCIS_or_LCIS_type','T_category','ER','PR','HER2','HER2_IHC','HER2_SISH',"BRCA_mutation"
    # 0 : 5개 제거(location of cancers,ER,PR,HER2_IHC,BRCA_mutation)
    #'dx', 'number of cancers', 'NG','HG','HG_score_1','HG_score_2','HG_score_3','DCIS_or_LCIS','DCIS_or_LCIS_type','T_category','HER2','HER2_SISH'
    # 1 : 3개 제거(location of cancers,ER,BRCA_mutation)
    'dx', 'number of cancers', 'NG','HG','HG_score_1','HG_score_2','HG_score_3','DCIS_or_LCIS','DCIS_or_LCIS_type','T_category','HER2','HER2_SISH','PR','HER2_IHC'
    # 2 : 2개 제거((location of cancers,ER)
    #'dx', 'number of cancers', 'NG','HG','HG_score_1','HG_score_2','HG_score_3','DCIS_or_LCIS','DCIS_or_LCIS_type','T_category','HER2','HER2_SISH','PR','HER2_IHC','BRCA_mutation'
]


for col_name in cat_col_names:
    cat_values = np.unique(train_df[col_name])
    
    #print(f"{col_name}: {cat_values}\n")


# 수치형 변수
num_col_names = [
    'age','size of cancers','ER_Allred_score','PR_Allred_score','KI-67_LI_percent','HER2_SISH_ratio'
]

feature_columns = (
    num_col_names + cat_col_names + target_name)

train_df = train_df.loc[:,feature_columns]


test = test_df.loc[:,num_col_names + cat_col_names]


X = train_df.loc[:,num_col_names+cat_col_names]
y = train_df.loc[:,target_name]


In [5]:
import re
df = train_df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   age                1000 non-null   int64  
 1   sizeofcancers      1000 non-null   float64
 2   ER_Allred_score    1000 non-null   float64
 3   PR_Allred_score    1000 non-null   float64
 4   KI67_LI_percent    1000 non-null   float64
 5   HER2_SISH_ratio    1000 non-null   float64
 6   dx                 1000 non-null   int64  
 7   numberofcancers    1000 non-null   int64  
 8   NG                 1000 non-null   float64
 9   HG                 1000 non-null   float64
 10  HG_score_1         1000 non-null   float64
 11  HG_score_2         1000 non-null   float64
 12  HG_score_3         1000 non-null   float64
 13  DCIS_or_LCIS       1000 non-null   int64  
 14  DCIS_or_LCIS_type  1000 non-null   float64
 15  T_category         1000 non-null   float64
 16  HER2               1000 n

In [6]:
from pycaret.datasets import get_data 
from pycaret.classification import *
from pycaret.utils import check_metric
setup_clf = setup(data=df, target='N_category') # x,y 값을 정의

 
Setup Succesfully Completed!


,Description,Value
0,session_id,1974
1,Target Type,Binary
2,Label Encoded,None
3,Original Data,"(1000, 21)"
4,Missing Values,False
5,Numeric Features,14
6,Categorical Features,6
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [7]:
# ML 모델 성능 비교
# 기본
top = compare_models() # setup 하면 자동적으로 설정됨 compare
top

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,Gradient Boosting Classifier,0.783900,0.854700,0.799400,0.788600,0.791700,0.567200
1,CatBoost Classifier,0.781000,0.856400,0.793700,0.784600,0.788200,0.561500
2,Extreme Gradient Boosting,0.775300,0.848900,0.782500,0.782500,0.781400,0.550200
3,Extra Trees Classifier,0.774000,0.854400,0.777100,0.784000,0.779200,0.547600
4,Ridge Classifier,0.762400,0.000000,0.760200,0.778000,0.766600,0.524600
5,Random Forest Classifier,0.761100,0.834300,0.729400,0.793500,0.757300,0.522600
6,Linear Discriminant Analysis,0.761000,0.836900,0.757400,0.777300,0.764800,0.521800
7,Logistic Regression,0.759600,0.830700,0.760200,0.772400,0.762800,0.519000
8,Light Gradient Boosting Machine,0.756700,0.843700,0.763100,0.766600,0.762800,0.513000
9,Naive Bayes,0.743800,0.797700,0.779600,0.736500,0.756700,0.486400


In [20]:
# ML 모델 성능 비교
## 연관성 x 제거 버전
# 0. 5개 제거
top = compare_models() # setup 하면 자동적으로 설정됨 compare
top

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,CatBoost Classifier,0.779600,0.858300,0.799200,0.778600,0.787800,0.558500
1,Extreme Gradient Boosting,0.776700,0.845900,0.777100,0.788100,0.781100,0.553300
2,Light Gradient Boosting Machine,0.775400,0.847200,0.782700,0.784400,0.781900,0.550300
3,Gradient Boosting Classifier,0.773900,0.854100,0.782600,0.780000,0.780100,0.547400
4,Logistic Regression,0.769600,0.834300,0.765700,0.783700,0.771800,0.539100
5,Extra Trees Classifier,0.766800,0.847000,0.779800,0.771300,0.774500,0.533000
6,Random Forest Classifier,0.756800,0.822900,0.746400,0.773000,0.758500,0.513700
7,Naive Bayes,0.746700,0.797800,0.776800,0.741800,0.758100,0.492300
8,Ada Boost Classifier,0.746700,0.823800,0.782700,0.741900,0.760400,0.492200
9,Linear Discriminant Analysis,0.746700,0.834900,0.749000,0.757100,0.750700,0.493100


In [26]:
# ML 모델 성능 비교
## 연관성 x 제거 버전
# 1. 3개 제거
top = compare_models() # setup 하면 자동적으로 설정됨 compare
top

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,Gradient Boosting Classifier,0.783900,0.854700,0.799400,0.788600,0.791700,0.567200
1,CatBoost Classifier,0.781000,0.856400,0.793700,0.784600,0.788200,0.561500
2,Extreme Gradient Boosting,0.775300,0.848900,0.782500,0.782500,0.781400,0.550200
3,Extra Trees Classifier,0.774000,0.854400,0.777100,0.784000,0.779200,0.547600
4,Ridge Classifier,0.762400,0.000000,0.760200,0.778000,0.766600,0.524600
5,Random Forest Classifier,0.761100,0.834300,0.729400,0.793500,0.757300,0.522600
6,Linear Discriminant Analysis,0.761000,0.836900,0.757400,0.777300,0.764800,0.521800
7,Logistic Regression,0.759600,0.830700,0.760200,0.772400,0.762800,0.519000
8,Light Gradient Boosting Machine,0.756700,0.843700,0.763100,0.766600,0.762800,0.513000
9,Naive Bayes,0.743800,0.797700,0.779600,0.736500,0.756700,0.486400


In [32]:
# ML 모델 성능 비교
## 연관성 x 제거 버전
# 2. 2개 제거
top = compare_models() # setup 하면 자동적으로 설정됨 compare
top

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,Gradient Boosting Classifier,0.779600,0.854400,0.791000,0.786100,0.786800,0.558700
1,Extreme Gradient Boosting,0.775300,0.848900,0.782500,0.782500,0.781400,0.550200
2,CatBoost Classifier,0.773800,0.857700,0.788100,0.776600,0.781200,0.547200
3,Extra Trees Classifier,0.766800,0.850200,0.774400,0.773700,0.772300,0.533300
4,Random Forest Classifier,0.766700,0.845400,0.746200,0.790500,0.766100,0.533700
5,Linear Discriminant Analysis,0.762400,0.836700,0.760200,0.778000,0.766500,0.524600
6,Ridge Classifier,0.761000,0.000000,0.757400,0.777600,0.764700,0.521800
7,Logistic Regression,0.759600,0.833400,0.757400,0.773800,0.762100,0.519000
8,Light Gradient Boosting Machine,0.756700,0.843700,0.763100,0.766600,0.762800,0.513000
9,Ada Boost Classifier,0.739500,0.817100,0.771500,0.739200,0.753300,0.477900


In [ ]:
!pip install pycaret==2.0.0
!pip install scikit-learn==0.23.2

In [38]:
# pycaret 버전
import pycaret
#print('PyCaret: %s' % pycaret.__version__)

In [39]:
from pandas import read_csv

# pycaret classification 모듈
from pycaret.classification import setup

# pycaret classification 모델들을 비교하기 위한 모듈
from pycaret.classification import compare_models

# 컬럼 갯수
n_cols = df.shape[1]

# 컬럼명 String으로 반환
df.columns = [str(i) for i in range(n_cols)]
# 데이터셋 설정 -> 데이터 target
#grid = setup(data=df, target=df.columns[-1],html= True silent=True, verbose=False)
# 모델 학습
best = compare_models()
# best 모델 확인
print(best)

# baysian

In [6]:
import os
from sklearn.datasets import fetch_covtype
import random
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
#import category_encoders as ce
%load_ext autoreload
%autoreload 2

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # statistical data visualization
import os
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier


In [7]:
len(X),len(y)

(1000, 1000)

In [ ]:
!pip3 install bayesian-optimization

In [8]:
from sklearn.metrics import r2_score, mean_squared_error,f1_score

# MAPE 
def mean_absolute_percentage_error(y_test, y_pred):
    y_test, y_pred = np.array(y_test), np.array(y_pred)
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100

# 탐색 대상 함수 
def detect_cv(max_depth,learning_rate, n_estimators,max_features):
    model = gb_clf = GradientBoostingClassifier(n_estimators=int(n_estimators), 
                                                learning_rate=learning_rate, 
                                                max_features=int(max_features), 
                                                max_depth=int(max_depth), 
                                                random_state=42,
                                                #n_jobs= -1 # 전체값 = 모든 core 사용
                                                )

    # 모델 훈련
    model.fit(X_train, y_train)

    # 예측값 출력
    y_pred= model.predict(X_test)
    f1 = f1_score(y_test, y_pred)

    # 오차 최적화로 사용할 metric 반환 (베이지안은 이것을 성능수치로 받아들이게됨)
    return f1

In [9]:
#  bayesian-optimization 라이브러리의 BayesianOptimization 클래스 import
from bayes_opt import BayesianOptimization
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


In [11]:
# 실험해보고자하는 hyperparameter 집합
# max_depth(int, default: 3): 기본 학습자를 위한 최대 트리 깊이
# learning_rate(float, default: 0.1) : Boosting 학습률
# n_estimators(int, default: 100) : fit하기 위한 Boosted tree의 수

pbounds = {'max_depth': (1, 3),
           'max_features':(1,20),
            'learning_rate': (0.0001, 0.2), # 범위로 찾아보겠다
            'n_estimators': (100, 500) # 범위로 찾아보겠다
            }


# Bayesian optimization 객체 생성
# f : 탐색 대상 함수, pbounds : hyperparameter 집합
# verbose = 2 항상 출력, verbose = 1 최댓값일 때 출력, verbose = 0 출력 안함
# 함수 정의 ...퍼포먼스 인지... 몇번 반복... 최적 하이퍼 파라미터 찾는다.
bo=BayesianOptimization(f=detect_cv, pbounds=pbounds, verbose=2, random_state=42 )    

# 메소드를 이용해 최대화 과정 수행
# init_points :  초기 Random Search 갯수
# n_iter : 반복 횟수 (몇개의 입력값-함숫값 점들을 확인할지! 많을 수록 정확한 값을 얻을 수 있다.)
# acq : Acquisition Function들 중 Expected 
# Improvement(EI) 를 사용
# xi : exploration 강도 (기본값은 0.0)
bo.maximize(init_points=2, n_iter=500, acq='ei', xi=0.01)

# ‘iter’는 반복 회차, ‘target’은 목적 함수의 값, 나머지는 입력값을 나타냄. 
# 현재 회차 이전까지 조사된 함숫값들과 비교하여, 현재 회차에 최댓값이 얻어진 경우, 
# bayesian-optimization 라이브러리는 이를 자동으로 다른 색 글자로 표시하는 것을 확인.

# 찾은 파라미터 값 확인
print(bo.max)


|   iter    |  target   | learni... | max_depth | max_fe... | n_esti... |
-------------------------------------------------------------------------
| 1         | 0.7601    | 0.07497   | 2.901     | 14.91     | 339.5     |
| 2         | 0.7823    | 0.03129   | 1.312     | 2.104     | 446.5     |
| 3         | 0.7546    | 0.09304   | 2.341     | 19.33     | 499.7     |
| 4         | 0.7601    | 0.2       | 1.0       | 1.0       | 100.0     |
| 5         | 0.7658    | 0.156     | 1.215     | 1.045     | 243.0     |
| 6         | 0.7639    | 0.003321  | 2.14      | 4.345     | 444.3     |
| 7         | 0.763     | 0.2       | 1.0       | 1.0       | 500.0     |
| 8         | 0.6667    | 0.0001    | 1.0       | 1.0       | 318.2     |
| 9         | 0.7895    | 0.2       | 1.0       | 20.0      | 178.0     |
| 10        | 0.791     | 0.2       | 1.0       | 20.0      | 241.6     |
| 11        | 0.7658    | 0.2       | 1.0       | 1.0       | 191.4     |
| 12        | 0.764     | 0.2       | 

In [12]:
bo.maximize(init_points=2, n_iter=1000, acq='ei', xi=0.01)

# ‘iter’는 반복 회차, ‘target’은 목적 함수의 값, 나머지는 입력값을 나타냄. 
# 현재 회차 이전까지 조사된 함숫값들과 비교하여, 현재 회차에 최댓값이 얻어진 경우, 
# bayesian-optimization 라이브러리는 이를 자동으로 다른 색 글자로 표시하는 것을 확인.

# 찾은 파라미터 값 확인
print(bo.max)


|   iter    |  target   | learni... | max_depth | max_fe... | n_esti... |
-------------------------------------------------------------------------
| 503       | 0.7681    | 0.1203    | 2.416     | 1.391     | 488.0     |
| 504       | 0.7715    | 0.1665    | 1.425     | 4.455     | 173.4     |
| 505       | 0.7749    | 0.03747   | 1.042     | 1.11      | 347.3     |
| 506       | 0.7789    | 0.01697   | 2.984     | 1.253     | 117.5     |
| 507       | 0.7794    | 0.04387   | 1.005     | 8.076     | 277.4     |
| 508       | 0.7744    | 0.1445    | 1.034     | 4.83      | 123.9     |
| 509       | 0.764     | 0.1887    | 2.617     | 12.89     | 273.0     |
| 510       | 0.7985    | 0.1281    | 1.051     | 19.79     | 355.2     |
| 511       | 0.794     | 0.1377    | 1.113     | 14.73     | 326.4     |
| 512       | 0.7985    | 0.119     | 2.982     | 5.936     | 124.5     |
| 513       | 0.794     | 0.1807    | 1.015     | 19.85     | 257.0     |
| 514       | 0.7658    | 0.1826    | 